In [ ]:
from gsheets import Sheets
import pandas as pd
import numpy as np
import datetime as dt
import time
# import filepaths
# import content_feedback_report
# import schedule_methods
# import lexi_cobra_sync
import pygsheets
import pandas as pd
import numpy as np
import datetime as dt
import time
# import schedule_methods
import math
import datetime
from datetime import timedelta  
import emoji
import re
import warnings
import statistics
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
import infrastructure 
from calendar import monthrange
from dateutil.relativedelta import relativedelta
import xlsxwriter
import send_email
from colorama import Fore, Style
import filepath

### Load in Cobra SMS and Lexi SMS

In [ ]:
#Use Infrastructure.py that Lili has written
mamba = infrastructure.get_mamba()
lexi_sms = pd.read_csv(filepath.input_folder + 'SS_LC_merged_data.csv')

In [ ]:
lexi_sms = infrastructure.transform_sms_df(lexi_sms)
lexi_sms[['Hitpath Offer ID']] = lexi_sms[['Hitpath Offer ID']].apply(pd.to_numeric, errors='coerce')
lexi_sms.dropna(subset=['Hitpath Offer ID'], inplace = True)

### Load in SMS Content Warehouse (El Nino) and Content Testing Submissions Smartsheet

In [ ]:
ct =infrastructure.get_smartsheet('content_sms')

# Create a new function in scheduling methods
#Use this link: https://docs.google.com/spreadsheets/d/1obszkCQoE0ELOR1O0CrLVETUEmEIWlGuyAmK3FgWSJg/edit#gid=1099746391

cw = infrastructure.get_lanina()
cw.dropna(subset=['Reporting Content ID'],inplace=True)
cw[['OfferIDs']] = cw[['OfferIDs']].apply(pd.to_numeric, errors='coerce')
cw.dropna(subset=['OfferIDs'], inplace = True)

In [ ]:
mask = cw['Type (Pitch)'] == 'HOL'
cw = cw[~mask]

In [ ]:
# cw.to_csv('checkcw.csv')

### Load in Offers Smartsheet

In [ ]:
offers = infrastructure.get_smartsheet('offers_sms')
offers.rename(columns = {'Hitpath Offer ID':'Hitpath ID'}, inplace = True)
offers.dropna(subset=['Hitpath ID'],inplace=True)
offers['Hitpath ID'] = offers['Hitpath ID'].astype(int)

In [ ]:
cw['Allocation Period (Date Added)'] = pd.to_datetime(cw['Allocation Period (Date Added)'], format='mixed')
#cw = cw[~(cw['Content ID'].str.contains('HTML'))]
cw.dropna(subset=['Reporting Content ID'],inplace=True)

In [ ]:
ct['Submission Date'] = pd.to_datetime(ct['Submission Date'])
ct.rename(columns = {'Date Added to La Nina (Akshad)':'Date Added to La Nina'}, inplace = True)
ct['Date Added to La Nina'].replace(to_replace=r"[a-zA-Z- ()]", value="", regex=True, inplace=True)
ct['Date Added to La Nina'] = pd.to_datetime(ct['Date Added to La Nina'], errors='coerce')
ct.rename(columns={"Hitpath ID": "HitPath ID"},inplace=True)

In [ ]:
mamba['Affiliate ID']=mamba['Dataset'].str.split('_').str[2]
mamba[['Limit','Offset']] = mamba[['Limit','Offset']].fillna("")
mamba['Drop Number'] = mamba['Drop'].str.split(expand=True)[1].astype(int)
mamba['Day of Week'] = mamba['Date'].dt.dayofweek
mamba.rename(columns={"Segment ": "Segment"},inplace=True)

In [ ]:
lexi_sms['eCPM'] = lexi_sms['Revenue']*1000 / lexi_sms['Delivered']
lexi_sms = lexi_sms[(lexi_sms['Date'].dt.date >= (date.today() + timedelta(days=-30)))]

In [ ]:
p_vert_ecpms = lexi_sms[lexi_sms['Send Strategy']=='P'].groupby(['Data Vertical','DP&Pub']).agg(Revenue=('Revenue',sum), Delivered=('Delivered',sum), Drops=('Hitpath Offer ID','count')).reset_index().rename(columns={'Drops':'Recent Dataset Vertical Drops'})
p_vert_ecpms['Recent Dataset Vertical eCPM'] = p_vert_ecpms['Revenue']*1000 / p_vert_ecpms['Delivered']

In [ ]:
lexi_sms = pd.merge(lexi_sms, p_vert_ecpms[['Data Vertical','DP&Pub','Recent Dataset Vertical eCPM','Recent Dataset Vertical Drops']], how='left', left_on=['Data Vertical','DP&Pub'], right_on=['Data Vertical','DP&Pub'])

p_ecpms = lexi_sms[lexi_sms['Send Strategy']=='P'].groupby(['DP&Pub']).agg(Revenue=('Revenue',sum), Delivered=('Delivered',sum)).reset_index()
p_ecpms['Recent Dataset P eCPM'] = p_ecpms['Revenue']*1000 / p_ecpms['Delivered']
lexi_sms = pd.merge(lexi_sms, p_ecpms[['DP&Pub','Recent Dataset P eCPM']], how='left', left_on=['DP&Pub'], right_on=['DP&Pub'])

In [ ]:
ccid_tests = lexi_sms[(lexi_sms['Send Strategy'].isin(['CT','PT','OT']))]

In [ ]:
def recent_pass(row):
        if row['Recent Dataset Vertical Drops'] > 3:
            if row['eCPM'] > row['Recent Dataset Vertical eCPM']:
                return True
            else:
                return False
        else:
            if row['eCPM'] > row['Recent Dataset P eCPM']: 
                return True
            else:
                return False

In [ ]:
ccid_tests['Pass'] = ccid_tests[['eCPM','Recent Dataset Vertical eCPM','Recent Dataset Vertical Drops','Recent Dataset P eCPM']].apply(lambda x: recent_pass(x), axis=1)
# ccid_tests.to_csv('ccidtestsauditupdated.csv')

In [ ]:
ccid_tests

In [ ]:
#missing opportunity cost
#test dates replaced with date
ccid_tests['Data Vertical'].fillna('Error Finding Vertical', inplace=True)
ccid_tests['Hitpath Offer ID'] = ccid_tests['Hitpath Offer ID'].astype("Int64")
ccid_tests_g = ccid_tests.groupby(['Creative Type'], as_index=False).agg(
        Revenue = ('Revenue',sum),
        Delivered = ('Delivered',sum),
        Clicks = ('Clicks',sum),
        #Opportunity_Cost = ('Opportunity Cost',sum),
        Vertical = ('Data Vertical', lambda x: x.value_counts().index[0]),
        Pass = ('Pass', sum),
        Test_Dates = ('Date', list ),
        HitPath = ('Hitpath Offer ID', lambda x: x.value_counts().index[0])
).set_index('HitPath').sort_index()
ccid_tests_g.index.rename('HitPath ID',inplace=True)
ccid_tests_g

In [ ]:
#no opportunity cost
ccid_tests_g['Test Dates'] = ccid_tests_g['Test_Dates'].apply(lambda x: str([d.strftime("%D") for d in x])[1:-1].replace("'",'') )
ccid_tests_g['eCPM'] = ccid_tests_g['Revenue']*1000 / ccid_tests_g['Delivered']
ccid_tests_g['CTR'] = ccid_tests_g['Clicks'] / ccid_tests_g['Delivered']
ccid_tests_g['Passed Testing'] = ccid_tests_g['Pass'].apply(lambda x: True if x>0 else False)
ccid_tests_g.rename(columns={'Creative Type':'CC ID',
                                 'eCPM':'eCPM from All Tests',
                                 'CTR':'CTR from All Tests',
                                 #'Opportunity_Cost':'Opportunity Cost from All Tests',
                                 'Revenue':'Revenue from All Tests',
                                 'Delivered':'Delivered from All Tests'},
                        inplace=True)
ccid_tests_g = ccid_tests_g[['CC ID','Test Dates','eCPM from All Tests','CTR from All Tests',#'Opportunity Cost from All Tests',
                   'Revenue from All Tests','Delivered from All Tests','Passed Testing']]
ccid_tests_g = ccid_tests_g[ccid_tests_g['CC ID']!='HTML']
passed_ids = ccid_tests_g[ccid_tests_g['Passed Testing']==True].drop(columns='Passed Testing')
failed_ids = ccid_tests_g[ccid_tests_g['Passed Testing']==False].drop(columns='Passed Testing')
passed_ids['Test Dates'] = passed_ids['Test Dates'].str.replace(',',"\n").str.replace(" ","")
failed_ids['Test Dates'] = failed_ids['Test Dates'].str.replace(',',"\n").str.replace(" ","")

In [ ]:
def parse_pieces(row):
    if row is np.nan:
        return row
    else:
        try:
            return int(row)
        except:
            row = re.sub("[a-zA-Z &,!.]", "", row)
            row = row.split('-')[-1]
            if (len(row) <=2) & (len(row) > 0):
                return int(row)
            else:
                return np.nan 

In [ ]:
ct[['HitPath ID']] = ct[['HitPath ID']].apply(pd.to_numeric, errors='coerce')
ct['HitPath ID'] = ct['HitPath ID'].astype("Int64")
ct.dropna(subset=['HitPath ID'], inplace = True)
requested_month_df = ct[ct['Submission Date'].dt.date >= (date.today() + timedelta(days=-30))]

approved_month_df = cw[cw['Allocation Period (Date Added)'].dt.date >= (date.today() + timedelta(days=-30))]

requested_month_df['HitPath ID'] = requested_month_df['HitPath ID'].astype("Int64")

requested_month_df['Number of Pieces Requested Per Shortcode'] = requested_month_df['# of Pieces Requested Per Shortcode'].apply(lambda x: parse_pieces(x) )
requested_pieces = requested_month_df.groupby('HitPath ID').agg({'Number of Pieces Requested Per Shortcode':'sum'}  ) 

approved_month_df['OfferIDs'] = approved_month_df['OfferIDs'].astype("Int64")
approved_month_df = approved_month_df.explode('OfferIDs') 
approved_month_df.rename(columns={"OfferIDs": "HitPath ID"},inplace=True)
approved_pieces = approved_month_df.groupby('HitPath ID').agg({'Reporting Content ID':'nunique'}) 
approved_pieces.rename(columns={"Reporting Content ID": "Number of Pieces Approved"},inplace=True)
  
pieces_in_testing_df = lexi_sms[(lexi_sms['Date'].dt.date >= (date.today() + timedelta(days=-30))) & (lexi_sms['Send Strategy']=='CT')]
#no creative type available
pieces_in_testing_df['HitPath ID'] = pieces_in_testing_df['Hitpath Offer ID']
pieces_in_testing_df['HitPath ID'] = pieces_in_testing_df['HitPath ID'].astype("Int64")
pieces_in_testing_df = pieces_in_testing_df.explode('HitPath ID') 
pieces_in_testing = pieces_in_testing_df.groupby('HitPath ID').agg({'Creative Type':'nunique'})
pieces_in_testing.rename(columns={"Creative Type": "Number of Pieces Tested"},inplace=True) 

passed_testing_pieces = passed_ids.groupby('HitPath ID').agg({'CC ID':'nunique'})
passed_testing_pieces.rename(columns={"CC ID": "Number of Pieces Passed Testing"},inplace=True)  

failed_testing_pieces = failed_ids.groupby('HitPath ID').agg({'CC ID':'nunique'})
failed_testing_pieces.rename(columns={"CC ID": "Number of Pieces Failed Testing"},inplace=True) 

content_ids = approved_month_df.groupby('HitPath ID').agg({'Reporting Content ID': lambda x: '\n'.join(x)})
content_ids.rename(columns={"Reporting Content ID": "Approved Content IDs"},inplace=True) 

rolling_thirty = pd.concat([content_ids, approved_pieces, pieces_in_testing, requested_pieces, passed_testing_pieces, failed_testing_pieces], axis=1, join = 'outer')
rolling_thirty['Percent of Pieces Passed'] = rolling_thirty['Number of Pieces Passed Testing'] / rolling_thirty['Number of Pieces Tested']

smartsheet_offers = set(offers['Hitpath ID'])
rolling_offers = set(rolling_thirty.index)
invalid_offers_rolling = list(rolling_offers.difference(smartsheet_offers))
rolling_thirty.drop(invalid_offers_rolling, inplace=True)

In [ ]:
#remove content id, merge numeric values, groupby pieces approved/tested

In [ ]:
# rolling_thirty.to_csv('checkrollingthirty.csv')

In [ ]:
rolling_cw_df = cw[(cw['Allocation Period (Date Added)'].dt.date >= (date.today() + timedelta(days=-30)))] #&(cw['Content Approval Status'].isin(['Approved - Passed']))]
rolling_cw_df = rolling_cw_df.explode('OfferIDs') 
rolling_cw_df.rename(columns={"OfferIDs": "HitPath ID"},inplace=True)
rolling_cw_df.rename(columns={"Type": "Shortcode"},inplace=True)

In [ ]:
lexi_sms['Hitpath Offer ID'] = lexi_sms['Hitpath Offer ID'].astype(int)

In [ ]:
#no body content in schedule
def not_seen(cc_id):
    upcoming_mamba = mamba[(mamba['Date'].dt.date>=date.today() ) & ((mamba['Creative'].str.contains(cc_id)))]
    lexi_cc_id = lexi_sms[lexi_sms['Creative Type']==cc_id]
    if (len(upcoming_mamba)==0) & (len(lexi_cc_id)==0):
        return True
    else:
        return False

In [ ]:
# rolling_cw_df.to_csv('cwaudit.csv')

In [ ]:
# rolling_cw_df = rolling_cw_df[~rolling_cw_df['Content Approval Status'].str.contains('Not')]
rolling_cw_df = rolling_cw_df[rolling_cw_df['Content Approval Status'].str.contains('Live')]


In [ ]:
rolling_cw_df.drop(rolling_cw_df[rolling_cw_df['Reporting Content ID'].apply(lambda x: not_seen(x)==False)].index, inplace=True)
awaiting_testing_pieces = rolling_cw_df.groupby(['HitPath ID','Shortcode'], as_index = False).agg({'Reporting Content ID':'nunique', 'Reporting Content ID':lambda x: '\n'.join(x),'Allocation Period (Date Added)':'max',})
awaiting_testing_pieces.rename(columns={"Reporting Content ID": "Number of Pieces Awaiting Testing"},inplace=True)

a = rolling_cw_df.groupby(['HitPath ID','Shortcode'], as_index = False).agg({'Reporting Content ID': 'nunique'})
a.rename(columns={"Reporting Content ID": "Number of Pieces Awaiting Testing"},inplace=True)
b = rolling_cw_df.groupby(['HitPath ID','Shortcode'], as_index = False).agg({'Allocation Period (Date Added)':'max','Reporting Content ID': lambda x: '\n'.join(x)})
b.rename(columns={"Reporting Content ID": "Content ID's"},inplace=True)

awaiting_testing_pieces = pd.merge(a, b, on =['HitPath ID','Shortcode'])
awaiting_testing_pieces = awaiting_testing_pieces.set_index('HitPath ID')

In [ ]:
# lexi_sms.to_csv('checkrev.csv')

In [ ]:
#no CPM or Operational Status

def find_new_hitpaths_to_highlight(df):
    dfc = df.copy()
    dfc.reset_index(inplace=True)
    indices = []
    for index, row in dfc.iterrows():
        hit = int(row['HitPath ID'])
        if len(lexi_sms[lexi_sms['Hitpath Offer ID']==hit]) == 0:
            indices.append(index+2)
    return indices

#find out the dataframe that meets this condition,applies specific color to that index that meets the criteria
#def find_cpm_to_highlight(df): #could combine with previous function to make slightly faster
    #dfc = df.copy()
    #dfc.reset_index(inplace=True)
    #indices = []
    #for index, row in dfc.iterrows():
       # if ('CPM' in row['Status']) or ('Only' in row['Status']):
             #   indices.append(index+2)
      #  return indices
        
def get_30_day_rev(hit):
    return lexi_sms[(lexi_sms['Date']>=lexi_sms['Date'].max()+timedelta(days=-30)) &(lexi_sms['Hitpath Offer ID']==int(hit))]['Revenue'].sum()

In [ ]:
awaiting_testing_pieces['AM'] = awaiting_testing_pieces.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['RX Rep'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
awaiting_testing_pieces['Offer Name'] = awaiting_testing_pieces.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Offer Name'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
awaiting_testing_pieces['Advertiser'] = awaiting_testing_pieces.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Advertiser'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
#awaiting_testing_pieces['Status Change Date'] = awaiting_testing_pieces.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Status Change Date'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')

#add shortcode to awaiting_testing pieces

awaiting_testing_pieces['30 Day Revenue'] = awaiting_testing_pieces.index.to_series().apply(get_30_day_rev)
awaiting_testing_pieces.sort_values(by='30 Day Revenue',ascending=False,inplace=True)

awaiting_testing_pieces['Status'] = awaiting_testing_pieces.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Status'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
awaiting_testing_pieces = awaiting_testing_pieces[['Shortcode',"Content ID's",'Offer Name','AM','Advertiser','Number of Pieces Awaiting Testing', 'Allocation Period (Date Added)', '30 Day Revenue','Status']]

#adjust based on sms
#address na = true or false if error message
awaiting_testing_pieces_paused=awaiting_testing_pieces[awaiting_testing_pieces['Status'].str.contains('Paused') | awaiting_testing_pieces['Status'].str.contains('Canceled')]
awaiting_testing_pieces_testing=awaiting_testing_pieces[awaiting_testing_pieces['Status'].str.contains('In Testing') | awaiting_testing_pieces['Status'].str.contains('Retest')]
awaiting_testing_pieces_live = awaiting_testing_pieces[awaiting_testing_pieces['Status'].str.contains('Live') | awaiting_testing_pieces['Status'].str.contains('Production')]
#awaiting_testing_pieces_pending=awaiting_testing_pieces[awaiting_testing_pieces['Status'].str.contains('Pending')]
#awaiting_testing_pieces_setup=awaiting_testing_pieces[awaiting_testing_pieces['Status'].str.contains('Set Up:')]

#removed setup due to no setup copy
awaiting_testing_new_hitpath_indices_live = find_new_hitpaths_to_highlight(awaiting_testing_pieces_live)
awaiting_testing_new_hitpath_indices_testing = find_new_hitpaths_to_highlight(awaiting_testing_pieces_testing)
awaiting_testing_new_hitpath_indices_paused = find_new_hitpaths_to_highlight(awaiting_testing_pieces_paused)
#awaiting_testing_new_hitpath_indices_pending = find_new_hitpaths_to_highlight(awaiting_testing_pieces_pending)
#awaiting_testing_new_hitpath_indices_setup = find_new_hitpaths_to_highlight(awaiting_testing_pieces_setup)

#awaiting_testing_cpm_indices_paused = find_cpm_to_highlight(awaiting_testing_pieces_paused)
#awaiting_testing_cpm_indices_testing = find_cpm_to_highlight(awaiting_testing_pieces_testing)
#awaiting_testing_cpm_indices_live = find_cpm_to_highlight(awaiting_testing_pieces_live)
#awaiting_testing_cpm_indices_cpm = find_cpm_to_highlight(awaiting_testing_pieces_cpm)
#awaiting_testing_cpm_indices_setup = find_cpm_to_highlight(awaiting_testing_pieces_setup)

    
upcoming_ct = mamba[(mamba['Date'].dt.date>=date.today()) & (mamba['Send Strategy']=='CT')]

upcoming_ct = upcoming_ct[['Offer','Creative','Date','Time','Dataset']]
upcoming_ct['Creative'].replace('',np.nan, inplace=True)
upcoming_ct.dropna(subset=['Offer','Creative'],inplace=True)
upcoming_ct['HitPath ID'] = upcoming_ct['Offer'].apply(lambda x: x.split('-')[0])
upcoming_ct.set_index('HitPath ID',inplace=True)
upcoming_ct.drop(columns='Offer', inplace=True)
upcoming_ct.rename(columns={"Creative": "Content IDs"},inplace=True)
upcoming_ct.sort_index(inplace=True)

In [ ]:
#no creative type
def cc_traffic(hit):
    hit = int(hit)
    last_90_hitpath = lexi_sms[(lexi_sms['Hitpath Offer ID']==hit) & (lexi_sms['Date'].dt.date >= (date.today() + timedelta(days=-90))  )]
    last_90_hitpath_cc = last_90_hitpath[last_90_hitpath['Creative Type']!='HTML']
    if len(last_90_hitpath) != 0:
        return 100*len(last_90_hitpath_cc) / len(last_90_hitpath)
    else:
        return np.nan

#no creative types
#rolling_thirty['Custom Content Traffic Percent'] = rolling_thirty.index.to_series().apply(lambda x: cc_traffic(x))
rolling_thirty.index.rename('HitPath ID',inplace=True)

def highlight_first_cc(df):
    dfc = df.copy()
    dfc.reset_index(inplace=True)
    indices = []
    for index, row in dfc.iterrows():
            
        hit = int(row['HitPath ID'])
        if len(lexi_sms[(lexi_sms['Hitpath Offer ID']==hit)])==0:
            indices.append(index+2)
    return indices

In [ ]:
rolling_thirty_indices = highlight_first_cc(rolling_thirty)
passed_indices = highlight_first_cc(passed_ids)
    
rolling_thirty['AM'] = rolling_thirty.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['RX Rep'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
rolling_thirty['Offer Name'] = rolling_thirty.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Offer Name'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
rolling_thirty['Advertiser'] = rolling_thirty.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Advertiser'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')

passed_ids['AM'] = passed_ids.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['RX Rep'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
passed_ids['Offer Name'] = passed_ids.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Offer Name'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
passed_ids['Advertiser'] = passed_ids.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Advertiser'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')

failed_ids['AM'] = failed_ids.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['RX Rep'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
failed_ids['Offer Name'] = failed_ids.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Offer Name'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
failed_ids['Advertiser'] = failed_ids.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Advertiser'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')

upcoming_ct['AM'] = upcoming_ct.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['RX Rep'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
upcoming_ct['Offer Name'] = upcoming_ct.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Offer Name'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')
upcoming_ct['Advertiser'] = upcoming_ct.index.to_series().apply(lambda x: offers[offers['Hitpath ID']==int(x)]['Advertiser'].values[0] if len(offers[offers['Hitpath ID']==int(x)])>=1 else '')


In [ ]:
def find_approval_date(ccid_list):
    ccid_list = ccid_list.strip()
    dates=[]
    for ccid in ccid_list.split('\n'):
        ccid = ccid.strip()
        ccid = ccid.replace('/','')
        try:
            dates.append((cw[cw['Reporting Content ID']==ccid]['Allocation Period (Date Added)'].values[0])) 
            return dates[0] #note this will return the date from the first ccid listed
        except:
            pass

In [ ]:
upcoming_ct['Approval Date'] = upcoming_ct['Content IDs'].apply(lambda x: find_approval_date(x))
recent_cw = cw.copy()
recent_cw['Allocation Period'] = recent_cw['Allocation Period (Date Added)'].dt.date
recent_cw = recent_cw[recent_cw['Allocation Period'] >= (date.today() + timedelta(days=-30))]

mamba_first_cc = mamba[mamba['Send Strategy'].isin(['CT','PT','OT'])]
mamba_first_cc = mamba_first_cc.replace('',np.nan)

mamba_first_cc = mamba_first_cc.replace('None','')
mamba_first_cc = mamba_first_cc.replace('nan','')
mamba_first_cc = mamba_first_cc.replace(np.nan,'')

mamba_first_cc['All Content Columns'] = mamba_first_cc['Creative'] #+ "\n" + mamba_first_cc['Content'] + "\n" + mamba_first_cc['MMID']
mamba_first_cc['All Content Columns'] = mamba_first_cc['All Content Columns'].str.replace('/','\n')
mamba_first_cc['All Content Columns'] = mamba_first_cc['All Content Columns'].apply(lambda x: str(x).split('\n') )
mamba_first_cc = mamba_first_cc.explode('All Content Columns')
mamba_first_cc['All Content Columns'] = mamba_first_cc['All Content Columns'].str.replace('"','')
mamba_first_cc['All Content Columns'] = mamba_first_cc['All Content Columns'].str.strip()
mamba_first_cc = mamba_first_cc.replace('',np.nan)

In [ ]:
mamba_first_cc = pd.merge(mamba_first_cc, cw[['Reporting Content ID','index']], how='left', left_on='All Content Columns', right_on='Reporting Content ID', suffixes=['_1','_ccid'])

mamba_first_cc = mamba_first_cc.replace(np.nan,'')

mamba_first_cc['Mamba Testing ID'] = mamba_first_cc[['All Content Columns','Reporting Content ID']].max(axis=1)

def extract_substring(text):
        return text.split('*')[0]  # Extract everything before the comma
    
mamba_first_cc['Mamba Testing ID'] = mamba_first_cc['Mamba Testing ID'].apply(extract_substring)

mamba_first_cc = mamba_first_cc.groupby(['Mamba Testing ID']).agg({'Date':'min'}).reset_index()

recent_cw = pd.merge(recent_cw[['Reporting Content ID','Allocation Period (Date Added)']], mamba_first_cc, how='left', left_on='Reporting Content ID', right_on='Mamba Testing ID')
recent_cw.drop(columns = 'Mamba Testing ID', inplace=True)
recent_cw = pd.merge(recent_cw, passed_ids.reset_index()[['CC ID']], left_on='Reporting Content ID', right_on='CC ID', how='left', indicator='Exist')
recent_cw.rename(columns={'Exist':'Passed Testing'},inplace=True)
recent_cw.drop(columns = 'CC ID', inplace=True)
recent_cw = pd.merge(recent_cw, failed_ids.reset_index()[['CC ID']], left_on='Reporting Content ID', right_on='CC ID', how='left', indicator='Exist')
recent_cw.rename(columns={'Exist':'Failed Testing','Date':'Earliest Test Date'},inplace=True)
recent_cw.drop(columns = 'CC ID', inplace=True)

recent_cw.loc[(recent_cw['Passed Testing'] == 'both', 'Test Result')] = 'Pass'
recent_cw.loc[(recent_cw['Failed Testing'] == 'both', 'Test Result')] = 'Fail'

recent_cw['Test Result'].fillna('Not yet Tested', inplace=True)

In [ ]:
from pandas.api.types import CategoricalDtype
sort_order = CategoricalDtype(['Not yet Tested', 'Pass', 'Fail'], ordered=True)
recent_cw['Test Result'] = recent_cw['Test Result'].astype(sort_order)
recent_cw.loc[((recent_cw['Test Result'] == 'Not yet Tested') & (recent_cw['Earliest Test Date']<lexi_sms['Date'].max()), 'High Alert')] = 1
recent_cw['High Alert'].fillna(0,inplace=True)

recent_cw.sort_values(by=['High Alert','Test Result','Earliest Test Date','Allocation Period (Date Added)'], inplace=True, ascending=[False,True,True,True], na_position='first')
recent_cw.set_index('Reporting Content ID',inplace=True)

#recent_cw['CCID in Awaiting Testing - Paused'] = recent_cw.index.to_series().apply(lambda x: awaiting_testing_pieces_paused['Content ID\'s'].str.contains(x).sum())
recent_cw['CCID in Awaiting Testing - Testing'] = recent_cw.index.to_series().apply(lambda x: awaiting_testing_pieces_testing['Content ID\'s'].str.contains(x).sum())
recent_cw['CCID in Awaiting Testing - Live'] = recent_cw.index.to_series().apply(lambda x: awaiting_testing_pieces_live['Content ID\'s'].str.contains(x).sum())

recent_cw_colors = recent_cw.reset_index()
#catch_all_awaiting_testing_paused_indices = list(recent_cw_colors[recent_cw_colors['CCID in Awaiting Testing - Paused']==1].index.values + 2)
catch_all_awaiting_testing_testing_indices = list(recent_cw_colors[recent_cw_colors['CCID in Awaiting Testing - Testing']==1].index.values + 2)
catch_all_awaiting_testing_live_indices = list(recent_cw_colors[recent_cw_colors['CCID in Awaiting Testing - Live']==1].index.values + 2)

recent_cw.drop(columns = ['Passed Testing','Failed Testing','High Alert','CCID in Awaiting Testing - Live','CCID in Awaiting Testing - Testing'], inplace = True) #'CCID in Awaiting Testing - Paused'], inplace=True)
    
recent_cw = recent_cw.reset_index()
recent_cw.drop_duplicates(subset='Reporting Content ID', inplace=True)

In [ ]:
#sort
awaiting_testing_pieces_live = awaiting_testing_pieces_live.sort_values(by=['Allocation Period (Date Added)','HitPath ID'], ascending=True)
awaiting_testing_pieces_testing = awaiting_testing_pieces_testing.sort_values(by=['Allocation Period (Date Added)','HitPath ID'], ascending=True)
awaiting_testing_pieces_paused = awaiting_testing_pieces_paused.sort_values(by=['Allocation Period (Date Added)','HitPath ID'], ascending=True)
rolling_thirty = rolling_thirty.sort_values(by=['HitPath ID'], ascending=True)
passed_ids = passed_ids.sort_values(by=['Test Dates','HitPath ID'], ascending=True)
failed_ids = failed_ids.sort_values(by=['Test Dates','HitPath ID'], ascending=True)
upcoming_ct = upcoming_ct.sort_values(by=['Date','HitPath ID'], ascending=True)
recent_cw = recent_cw.sort_values(by=['Allocation Period (Date Added)','Reporting Content ID'], ascending=True)

In [ ]:
rolling_thirty.index = rolling_thirty.index.astype(int)
passed_ids.index = passed_ids.index.astype(int)
failed_ids.index = failed_ids.index.astype(int)

In [ ]:
#new offer content to add to la nina
newcontent = pd.DataFrame(ct, columns =['HitPath ID','Unapproved Content Document Link (Copywriter)','Ready for Akshad (Kellie Checks; Alert Sent)','Date Added to La Nina','Offer Type'])
ctlink = "https://app.smartsheet.com/sheets/63665Vccw4XRFh8c7rGrrCf5mvG7gP5qvhCmx6J1?view=grid"
renamenewcontent = {
    'Unapproved Content Document Link (Copywriter)': 'Content Link',
    'Ready for Akshad (Kellie Checks; Alert Sent)': 'Ready Status'
}

newcontent.rename(columns=renamenewcontent, inplace=True)
newcontent.dropna(subset=['Content Link'], inplace=True)
waitinghitpaths = newcontent[(newcontent['Ready Status'] == True) & (newcontent['Date Added to La Nina'].isnull())]
waitinghitpaths['Link to Submission Sheet'] = ctlink
waitinghitpaths = waitinghitpaths.loc[:, ['HitPath ID','Offer Type','Content Link','Link to Submission Sheet']]
waitinghitpaths.sort_values(by=['HitPath ID'], ascending=True)
waitinghitpaths['HitPath ID'] = waitinghitpaths['HitPath ID'].astype(float)
waitinghitpaths.set_index('HitPath ID', inplace = True)

In [ ]:
waitinghitpaths

In [ ]:
awaiting_testing_pieces_paused

In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, Color, colors, fills
from openpyxl.utils.dataframe import dataframe_to_rows

from openpyxl.styles.borders import Border, Side, BORDER_THIN
thin_border = Border(
    left=Side(border_style=BORDER_THIN, color='00616161'),
    right=Side(border_style=BORDER_THIN, color='00616161'),
    top=Side(border_style=BORDER_THIN, color='00616161'),
    bottom=Side(border_style=BORDER_THIN, color='00616161')
)

In [ ]:
wb = Workbook()
ws = wb.active
awaiting_testing_pieces_live_sheet = wb['Sheet']
awaiting_testing_pieces_live_sheet.title = 'Awaiting Testing -  Live'
awaiting_testing_pieces_live.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
for r in dataframe_to_rows(awaiting_testing_pieces_live.reset_index(), index=False, header=True):
    awaiting_testing_pieces_live_sheet.append(r)

In [ ]:
awaiting_testing_pieces_testing.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
awaiting_testing_pieces_testing_sheet = wb.create_sheet('Awaiting Testing -  Testing')
for r in dataframe_to_rows(awaiting_testing_pieces_testing.reset_index(), index=False, header=True):
    awaiting_testing_pieces_testing_sheet.append(r)

In [ ]:
awaiting_testing_pieces_paused.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan    
awaiting_testing_pieces_paused_sheet = wb.create_sheet('Awaiting Testing -  Paused')
for r in dataframe_to_rows(awaiting_testing_pieces_paused.reset_index(), index=False, header=True):
    awaiting_testing_pieces_paused_sheet.append(r)

In [ ]:
waitinghitpaths.replace({np.nan: np.nan}, inplace = True)#Replace NaT with nan    
waitinghitpaths_sheet = wb.create_sheet('Content Not Added to La Nina')
for r in dataframe_to_rows(waitinghitpaths.reset_index(), index=False, header=True):
    waitinghitpaths_sheet.append(r)

In [ ]:
#awaiting_testing_pieces_cpm.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
#awaiting_testing_pieces_cpm_sheet = wb.create_sheet('Awaiting Testing -  CPM')
#for r in dataframe_to_rows(awaiting_testing_pieces_cpm.reset_index(), index=False, header=True):
    #awaiting_testing_pieces_cpm_sheet.append(r)

In [ ]:
#awaiting_testing_pieces_setup.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
#awaiting_testing_pieces_setup_sheet = wb.create_sheet('Awaiting Testing -  Setup')
#for r in dataframe_to_rows(awaiting_testing_pieces_setup.reset_index(), index=False, header=True):
    #awaiting_testing_pieces_setup_sheet.append(r)

In [ ]:
rolling_thirty.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
rolling_thirty_sheet = wb.create_sheet('Last 30 Days')
for r in dataframe_to_rows(rolling_thirty.reset_index(), index=False, header=True):
    rolling_thirty_sheet.append(r)

In [ ]:
passed_ids.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
passed_sheet = wb.create_sheet('Recent Passed IDs')
for r in dataframe_to_rows(passed_ids.reset_index(), index=False, header=True):
    passed_sheet.append(r)

In [ ]:
failed_ids.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan    
failed_sheet = wb.create_sheet('Recent Failed IDs')
for r in dataframe_to_rows(failed_ids.reset_index(), index=False, header=True):
    failed_sheet.append(r)

In [ ]:
upcoming_ct.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
upcoming_ct_sheet = wb.create_sheet('Upcoming Tests')
for r in dataframe_to_rows(upcoming_ct.reset_index(), index=False, header=True):
    upcoming_ct_sheet.append(r) 

In [ ]:
#     recent_cw.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
#     recent_cw_sheet = wb.create_sheet('All Recently Allocated CCIDs')
#     for r in dataframe_to_rows(recent_cw.reset_index(), index=False, header=True):
#         recent_cw_sheet.append(r)  

In [ ]:
recent_cw.replace({np.nan: np.nan}, inplace = True) #Replace NaT with nan
recent_cw_sheet = wb.create_sheet('All Recently Allocated CCIDs')
recent_cw_sheet.append([col for col in recent_cw.columns])
for row_pointer in range(len(recent_cw)):
    row = [recent_cw.iloc[row_pointer][col_pointer] for col_pointer in range(len(recent_cw.columns))]
    recent_cw_sheet.append(row) 

In [ ]:
for row_index in awaiting_testing_new_hitpath_indices_live:
    for cell in awaiting_testing_pieces_live_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        cell.border = thin_border
        awaiting_testing_pieces_live_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        awaiting_testing_pieces_live_sheet[('K'+str(row_index))].border = thin_border
    awaiting_testing_pieces_live_sheet[('K'+str(row_index))]='New Offer Highlighted Green'

In [ ]:
for row_index in awaiting_testing_new_hitpath_indices_testing:
    for cell in awaiting_testing_pieces_testing_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        cell.border = thin_border
        awaiting_testing_pieces_testing_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        awaiting_testing_pieces_testing_sheet[('K'+str(row_index))].border = thin_border
    awaiting_testing_pieces_testing_sheet[('K'+str(row_index))]='New Offer Highlighted Green'

In [ ]:
for row_index in awaiting_testing_new_hitpath_indices_paused:
    for cell in awaiting_testing_pieces_paused_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        cell.border = thin_border
        awaiting_testing_pieces_paused_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        awaiting_testing_pieces_paused_sheet[('K'+str(row_index))].border = thin_border
    awaiting_testing_pieces_paused_sheet[('K'+str(row_index))]='New Offer Highlighted Green'

In [ ]:
#for row_index in awaiting_testing_new_hitpath_indices_pending:
    #for cell in awaiting_testing_pieces_pending_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        #cell.border = thin_border
        #awaiting_testing_pieces_pending_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        #awaiting_testing_pieces_pending_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_pending_sheet[('K'+str(row_index))]='New Offer Highlighted Green'

In [ ]:
#removed temporarily due to no content in la nina-- NoneType object is not iterable
#for row_index in awaiting_testing_new_hitpath_indices_setup:
    #for cell in awaiting_testing_pieces_setup_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        #cell.border = thin_border
        #awaiting_testing_pieces_setup_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='9ef598'))
        #awaiting_testing_pieces_setup_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_setup_sheet[('K'+str(row_index))]='New Offer Highlighted Green'

In [ ]:
#for row_index in awaiting_testing_cpm_indices_live:
    #for cell in awaiting_testing_pieces_live_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #cell.border = thin_border
        #awaiting_testing_pieces_live_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #awaiting_testing_pieces_live_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_live_sheet[('K'+str(row_index))]='CPM Offer Highlighted Orange'

In [ ]:
#for row_index in awaiting_testing_cpm_indices_testing:
    #for cell in awaiting_testing_pieces_testing_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #cell.border = thin_border
        #awaiting_testing_pieces_testing_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #awaiting_testing_pieces_testing_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_testing_sheet[('K'+str(row_index))]='CPM Offer Highlighted Orange'

In [ ]:
#for row_index in awaiting_testing_cpm_indices_paused:
    #for cell in awaiting_testing_pieces_paused_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #cell.border = thin_border
        #awaiting_testing_pieces_paused_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #awaiting_testing_pieces_paused_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_paused_sheet[('K'+str(row_index))]='CPM Offer Highlighted Orange'

In [ ]:
#for row_index in awaiting_testing_cpm_indices_pending:
    #for cell in awaiting_testing_pieces_pending_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #cell.border = thin_border
        #awaiting_testing_pieces_pending_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #awaiting_testing_pieces_pending_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_pending_sheet[('K'+str(row_index))]='CPM Offer Highlighted Orange'

In [ ]:
#for row_index in awaiting_testing_cpm_indices_setup:
    #for cell in awaiting_testing_pieces_setup_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #cell.border = thin_border
        #awaiting_testing_pieces_setup_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        #awaiting_testing_pieces_setup_sheet[('K'+str(row_index))].border = thin_border
    #awaiting_testing_pieces_setup_sheet[('K'+str(row_index))]='CPM Offer Highlighted Orange'

In [ ]:
for row_index in rolling_thirty_indices:
    for cell in rolling_thirty_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='71AFE2'))
        cell.border = thin_border
        rolling_thirty_sheet[('L'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='71AFE2'))
        rolling_thirty_sheet[('L'+str(row_index))].border = thin_border
    rolling_thirty_sheet[('L'+str(row_index))]='First Custom Content for Offer Highlighted Blue'

In [ ]:
for row_index in passed_indices:
    for cell in passed_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='71AFE2'))
        cell.border = thin_border
        passed_sheet[('K'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='71AFE2'))
        passed_sheet[('K'+str(row_index))].border = thin_border
    passed_sheet[('K'+str(row_index))]='First Custom Content for Offer Highlighted Blue'

In [ ]:
for row_index in catch_all_awaiting_testing_testing_indices:
    for cell in recent_cw_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        cell.border = thin_border
        recent_cw_sheet[('E'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='f5b669'))
        recent_cw_sheet[('E'+str(row_index))].border = thin_border
    recent_cw_sheet[('E'+str(row_index))]='CCID in Awaiting Testing - Testing' 

In [ ]:
#for row_index in catch_all_awaiting_testing_paused_indices:
    #for cell in recent_cw_sheet[(str(row_index)):(str(row_index))]:
        #cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fc6a6a'))
        #cell.border = thin_border
        #recent_cw_sheet[('E'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='fc6a6a'))
        #recent_cw_sheet[('E'+str(row_index))].border = thin_border
    #recent_cw_sheet[('E'+str(row_index))]='CCID in Awaiting Testing - Paused' 

In [ ]:
for row_index in catch_all_awaiting_testing_live_indices:
    for cell in recent_cw_sheet[(str(row_index)):(str(row_index))]:
        cell.fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='3cb043'))
        cell.border = thin_border
        recent_cw_sheet[('E'+str(row_index))].fill = fills.PatternFill(patternType='solid', fgColor=Color(rgb='3cb043'))
        recent_cw_sheet[('E'+str(row_index))].border = thin_border
    recent_cw_sheet[('E'+str(row_index))]='CCID in Awaiting Testing - Live' 

In [ ]:
#removed awaiting_testing_pieces_cpm_sheet from all_sheets
#remove awaiting_testing_pieces_setup_sheet from all_sheets
all_sheets = [awaiting_testing_pieces_live_sheet,awaiting_testing_pieces_testing_sheet,awaiting_testing_pieces_paused_sheet,waitinghitpaths_sheet,rolling_thirty_sheet,passed_sheet,failed_sheet,upcoming_ct_sheet,recent_cw_sheet]

In [ ]:
for ws in all_sheets:
    dims = {}
    for row in ws.rows:
        for cell in ws["1:1"]:
            cell.font = Font(bold=True)
        for cell in row:
            newline_count = 1
            if cell.value:
                if type(cell.value)==str:
                    if ('\n' in cell.value):
                        newline_count = cell.value.count('\n')
                dims[cell.column_letter] = max((dims.get(cell.column_letter, 0), len(str(cell.value))))/newline_count 
    for col, value in dims.items():
        ws.column_dimensions[col].width = value

In [ ]:
import os

today = date.today().strftime("%m_%d_%Y")
save_path = filepath.output_folder + "Content Feedback Loop Reports/"
filename = os.path.join(save_path, f"Content_Feedback_Loop_Report_{today}.xlsx")

wb.save(filename)

#filename = f"Content_Feedback_Loop_Report_{today}.xlsx"
#wb.save(filename)

In [ ]:
#email
#toaddr = ['lili@rxmg.com', 'tinamarie@rxmg.com', 'g.chao@rxmg.com','nathan@rxmg.com','r.woodward@rxmg.com']
toaddr = ['g.chao@rxmg.com', 'lili@rxmg.com', 'nathan@rxmg.com','nina@rxmg.com','offernotices@rxmg.com', 'b.ratzlaff@rxmg.com']
#toaddr = ['g.chao@rxmg.com']
subject_line = f"SMS Team Content Feedback Loop Report - {today}"
email_body = "Hi Team,\n\n"
email_body += "Attached is today's Content Feedback Loop Report.\n" 
email_body += f"\nThanks,\n{filepath.name}\n\n"

#send email
for i in toaddr:
    send_email.send_email([filename], subject_line, email_body, i)